In [1]:
%gui qt

This notebook is an example of how to analyze the thickness of a layer of tissue.

## Generate mock data
First, we generate a set of mock data simulating a slab of tissue with three layers.

In [2]:
import numpy as np
from skimage import draw



In [3]:
tissue_x = 450
tissue_y = 450
tissue_z = 150

indent_diameter = 50




classified_tissue = 4 * np.ones((tissue_z, tissue_x, tissue_y))

# make the stroma
classified_tissue[50:125, :, :] = 2

# make the muscle
classified_tissue[125:, :, :] = 3

# make the epithelial layer
classified_tissue[25:50, :, :] = 1

sphere = draw.ellipsoid(indent_diameter, indent_diameter, indent_diameter)
half_sphere = sphere[52:102, :, :]

half_sphere_in_cube = 2 * np.ones(half_sphere.shape)
half_sphere_in_cube[half_sphere] = 1

edge_border_x = int((tissue_x - (2 * half_sphere.shape[1])) / 2)
mid_border_x = edge_border_x + half_sphere.shape[1]
end_border_x = mid_border_x + half_sphere.shape[1]
edge_border_y = int((tissue_y - (2 * half_sphere.shape[2])) / 2)
mid_border_y = edge_border_y + half_sphere.shape[2]
end_border_y = mid_border_y + half_sphere.shape[2]

classified_tissue[50:100, edge_border_x:mid_border_x, edge_border_y:mid_border_y] = half_sphere_in_cube
classified_tissue[50:100, mid_border_x:end_border_x, edge_border_y:mid_border_y] = half_sphere_in_cube
classified_tissue[50:100, edge_border_x:mid_border_x, mid_border_y:end_border_y] = half_sphere_in_cube
classified_tissue[50:100, mid_border_x:end_border_x, mid_border_y:end_border_y] = half_sphere_in_cube

In [4]:
%gui qt
import napari

In [5]:
colors = {
    0: [0, 0, 0, 0],
    1: [0.5, 0.79, 0.5, 1],
    2: [0.75, 0.68, 0.83, 1],
    3: [0.99, 0.75, 0.52, 1],
    4: [0, 0, 0, 1]
}
viewer = napari.view_labels(classified_tissue, color=colors, opacity=0.4)

In [6]:
from morphologyanalysis.volume.label import label_boundaries
edge_labels = label_boundaries(classified_tissue)

# add the edge label image to the napari viewer
viewer.add_labels(edge_labels, color=colors);

In [8]:
from morphologyanalysis.surface.mesh import mesh_from_interface

surf_bot = mesh_from_interface(label_im=edge_labels, parent_label=1, neighbor_label=4)
surf_top = mesh_from_interface(label_im=edge_labels, parent_label=2, neighbor_label=1)

In [10]:
from morphologyanalysis.surface.utils import find_interface

bot_interface = find_interface(label_im=edge_labels, parent_label=1, neighbor_label=2)

interface_im = np.zeros((150, 450, 450))
interface_im[bot_interface[:, 0], bot_interface[:,1], bot_interface[:, 2]] = 1

viewer.add_labels(interface_im)

<Labels layer 'interface_im [1]' at 0x7fac1f34ef50>

In [11]:
from skimage.measure import marching_cubes

# use the mask to help the meshing
verts, faces, normals, values = marching_cubes(interface_im, mask=interface_im.astype(np.bool))

In [15]:
viewer.add_surface((verts, faces, values))

<Surface layer 'Surface' at 0x7fabfdfbfb50>

In [15]:
surf_bot_n["distances"] = np.empty(surf_bot.n_points)
for i in range(surf_bot_n.n_points):
    p = surf_bot_n.points[i]
    vec = surf_bot_n["Normals"][i] * surf_bot_n.length
    p0 = p - vec
    p1 = p + vec
    ip, ic = surf_top.ray_trace(p0, p1, first_point=True)
    dist = np.sqrt(np.sum((ip - p)**2))
    surf_bot_n["distances"][i] = dist


14.51

In [17]:
p2 = pv.PlotterITK()
p2.add_mesh(surf_bot_n, scalars="distances", smooth_shading=True)
p2.add_mesh(surf_top, opacity=0.75, smooth_shading=True)
p2.show()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…